In [2]:
import pandas as pd
import numpy as np
import math
import random

from collections import namedtuple, deque
from itertools import count
from PIL import Image

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.utils.data as data_utils
import torchvision.transforms as T
from torch.autograd import Variable

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

import shutil
import os
import gc
import gym
import gym_test

In [3]:
#df = pd.read_csv('./dataset/clean.csv')
#shop_tag = df.pop('shop_tag')
#df.insert(52,'shop_tag', shop_tag)
#df.head()

,dt,chid,txn_cnt,txn_amt,domestic_offline_cnt,domestic_online_cnt,overseas_offline_cnt,overseas_online_cnt,domestic_offline_amt_pct,domestic_online_amt_pct,...,educd,trdtp,naty,poscd,cuorg,slam,gender_code,age,primary_card,shop_tag
0,1,10321418,3,3891.965283,3,0,0,0,1.00,0.00,...,4.0,5.0,1.0,99.0,30.0,95982.822967,1.0,4.0,1,45
1,1,10414574,2,10616.561549,2,0,0,0,1.00,0.00,...,2.0,15.0,1.0,2.0,30.0,130702.351368,1.0,3.0,1,15
2,1,10134567,2,23527.655416,0,2,0,0,0.00,1.00,...,3.0,11.0,1.0,3.0,30.0,112010.611717,0.0,4.0,1,48
3,1,10001003,9,17751.558260,7,2,0,0,0.75,0.25,...,3.0,11.0,1.0,2.0,30.0,59701.507360,0.0,3.0,1,48
4,1,10267183,1,21701.307598,1,0,0,0,1.00,0.00,...,6.0,15.0,1.0,99.0,30.0,NaN,0.0,6.0,0,2
5,1,10256467,1,2397.282233,1,0,0,0,1.00,0.00,...,4.0,2.0,1.0,1.0,30.0,95982.822967,0.0,5.0,1,12
6,1,10209279,1,5773.036485,1,0,0,0,1.00,0.00,...,3.0,8.0,1.0,4.0,30.0,100181.442972,1.0,4.0,1,33
7,1,10122747,3,13290.395882,1,2,0,0,0.16,0.84,...,5.0,5.0,1.0,3.0,30.0,68561.179121,0.0,4.0,1,48
8,1,10118615,1,5773.036485,1,0,0,0,1.00,0.00,...,3.0,11.0,1.0,2.0,30.0,87397.101503,0.0,5.0,1,48
9,1,10164338,2,9063.778202,2,0,0,0,1.00,0.00,...,4.0,15.0,1.0,99.0,30.0,59393.654415,1.0,3.0,1,15


In [2]:
env = gym.make('test-v0')

action_space: Discrete(49)
action_space_shape: ()
observation_space: Discrete(32975653)
observation_space_shape: Discrete(32975653)
data_shape (32975653, 53)
ACTION_LOOKUP_len: 49


In [3]:
print('State shape: ', env.observation_space.shape)
print('Number of actions: ', env.action_space.n)

State shape:  ()
Number of actions:  49


In [1]:
env.df_xy.head()

NameError: name 'env' is not defined

In [13]:
actions = []
rewards = []
count = 5

env.reset()
while True:
  action = env.action_space.sample()
  print(action)
  reward = env.step(action)
  print(reward)
  actions.append(action)
  rewards.append(reward)
  count -= 1
  if count==0:
    break

48
(dt                          1.200000e+01
chid                        1.008523e+07
txn_cnt                     4.000000e+00
txn_amt                     1.574194e+04
domestic_offline_cnt        4.000000e+00
domestic_online_cnt         0.000000e+00
overseas_offline_cnt        0.000000e+00
overseas_online_cnt         0.000000e+00
domestic_offline_amt_pct    1.000000e+00
domestic_online_amt_pct     0.000000e+00
overseas_offline_amt_pct    0.000000e+00
overseas_online_amt_pct     0.000000e+00
card_1_txn_cnt              0.000000e+00
card_2_txn_cnt              0.000000e+00
card_3_txn_cnt              0.000000e+00
card_4_txn_cnt              0.000000e+00
card_5_txn_cnt              0.000000e+00
card_6_txn_cnt              0.000000e+00
card_7_txn_cnt              0.000000e+00
card_8_txn_cnt              0.000000e+00
card_9_txn_cnt              0.000000e+00
card_10_txn_cnt             0.000000e+00
card_11_txn_cnt             0.000000e+00
card_12_txn_cnt             0.000000e+00
card_13_txn_

In [14]:
BUFFER_SIZE = int(1e5)  # replay buffer size
BATCH_SIZE = 64         # minibatch size
GAMMA = 0.99            # discount factor
TAU = 1e-3              # for soft update of target parameters
LR = 5e-4               # learning rate
UPDATE_EVERY = 4        # how often to update the network
#EPSILON = 0.8           # probability of chosing on-policy action

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
sc = StandardScaler()

In [7]:
'''
X = env.df_xy.iloc[:, :-1].values #每列資訊 除了最後一類
y = env.df_xy.iloc[:, -1].values #最後一類

sc = StandardScaler()
X = sc.fit_transform(X)
'''
'''
X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=0.1, random_state=1)

X_train = torch.from_numpy(X_train)
Y_train = torch.from_numpy(Y_train).double()

train = data_utils.TensorDataset(X_train, Y_train)
train_loader = data_utils.DataLoader(train, batch_size=BATCH_SIZE, shuffle=True)
'''

'\nX_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=0.1, random_state=1)\n\nX_train = torch.from_numpy(X_train)\nY_train = torch.from_numpy(Y_train).double()\n\ntrain = data_utils.TensorDataset(X_train, Y_train)\ntrain_loader = data_utils.DataLoader(train, batch_size=BATCH_SIZE, shuffle=True)\n'

In [6]:
X_train, X_test, Y_train, Y_test = train_test_split(sc.fit_transform(env.df_xy.iloc[:, :-1].values), env.df_xy.iloc[:, -1].values, test_size=0.1, random_state=1)

X_train = torch.from_numpy(X_train)
Y_train = torch.from_numpy(Y_train).double()

train_loader = data_utils.DataLoader(data_utils.TensorDataset(X_train, Y_train), batch_size=BATCH_SIZE, shuffle=True)

D:\Anaconda3\lib\site-packages\sklearn\utils\extmath.py:1015: RuntimeWarning: overflow encountered in square
  temp **= 2
D:\Anaconda3\lib\site-packages\sklearn\utils\extmath.py:1021: RuntimeWarning: overflow encountered in square
  new_unnormalized_variance -= correction ** 2 / new_sample_count
D:\Anaconda3\lib\site-packages\sklearn\utils\extmath.py:1021: RuntimeWarning: invalid value encountered in subtract
  new_unnormalized_variance -= correction ** 2 / new_sample_count
D:\Anaconda3\lib\site-packages\sklearn\preprocessing\_data.py:80: RuntimeWarning: overflow encountered in square
  upper_bound = n_samples * eps * var + (n_samples * mean * eps) ** 2


In [35]:
class DQN(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(52, 256)
        self.fc2 = nn.Linear(256, 64)
        self.fc3 = nn.Linear(64, 64)
        self.fc4 = nn.Linear(64, 128)
        self.fc5 = nn.Linear(128, 49)
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.dropout(x, p=0.25)
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        x = torch.sigmoid(self.fc5(x))
        return x

In [36]:
class Agent():
    def __init__(self, action_size, seed):
        self.action_size = action_size
        self.seed = random.seed(seed)


        # Q - Network
        self.qnet_local = DQN().double().to(device)
        self.qnet_target = DQN().double().to(device)

        self.optimizer = optim.Adam(self.qnet_local.parameters(), lr=0.001)

        self.memory = ReplayBuffer(action_size, BUFFER_SIZE, BATCH_SIZE, seed)

        self.t_step = 0
        self.train_loss = []

    def step(self, state, action, reward, next_state, done):
        #print(state.shape)
        self.memory.add(state, action, reward, next_state, done)
        # learn every 4 timesteps
        self.t_step = (self.t_step+1) % UPDATE_EVERY
        if self.t_step == 0:
            if len(self.memory) > BATCH_SIZE:
                experience = self.memory.sample()
                print('Experience sampled from memory : ', experience)
                self.learn(experience, GAMMA)


    def epsilon_greedy_action(self, state):
        state = state.to(device)
        self.qnet_local.eval()
        with torch.no_grad():
            action_values = self.qnet_local(state).max(1)[1]#.view(1, 1)
        self.qnet_local.train()

        if random.random() < 0.8:
            print('Predicted action based on QNetwork : ', action_values)
            return action_values.cpu().numpy()
            #return np.argmax(action_values.cpu().numpy())
        else:
            random_action = random.choices(np.arange(self.action_size), k=BATCH_SIZE)
            print('Chosing  random actions for the batch : ', random_action)
            return random_action
  
    def learn(self, experiences, gamma):
        #print('Started learning')
        states, actions, rewards, next_states, done = experiences#experiences[0].state, experiences[0].action, experiences[0].reward, experiences[0].next_state, experiences[0].done 
        criterion = torch.nn.BCELoss()
        self.qnet_local.train()
        self.qnet_target.eval()

        #predicted_targets = self.qnet_local(states)#.gather(1, actions)

        #print(next_states.view(1, 1))
        with torch.no_grad():
            labels_next = self.qnet_target(next_states).detach().max(1)[0].unsqueeze(1)

        #print('labels_next {}'.format(labels_next))

        labels = 0 + (gamma * labels_next)
        print("labels:", labels)
        predicted_targets = self.qnet_local(states).gather(1, actions.long())
        print("predicted_targets:", predicted_targets)
        #print("Predicted targets : {}, labels : {}".format(predicted_targets, labels))

        loss = criterion(predicted_targets, labels).to(device)
        print("===========================Training loss ============================")
        print(loss.item())
        self.train_loss.append(loss.item())
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()

        print('Total training losses : ', self.train_loss)

        # perform soft update
        self.soft_update(self.qnet_local, self.qnet_target, TAU)
  
    def soft_update(self, local_model, target_model, tau):
        for target_param, local_param in zip(target_model.parameters(), local_model.parameters()):
            target_param.data.copy_(tau*local_param.data + (1-tau)*target_param.data)

In [37]:
class ReplayBuffer():
    def __init__(self, action_size, buffer_size, batch_size, seed):
        self.action_size = action_size
        self.memory = deque(maxlen=buffer_size)
        self.batch_size = batch_size
        self.experiences = namedtuple("Experience", field_names=["state", "action", "reward", "next_state", "done"])
        self.seed = random.seed(seed)
  
    def add(self, state, action, reward, next_state, done):
        experience = self.experiences(state, action, reward, next_state, done)
        self.memory.append(experience)

    def sample(self):
        experiences = random.sample(self.memory, k=self.batch_size)
        #print('Experiences : ', len(experiences))
        '''
        for i, e in enumerate(experiences):
            print("e:", e.state)
            print("e:", len(e.state), i)
        '''
        states = torch.from_numpy(np.vstack([e.state for e in experiences if e is not None])).double().to(device)
        actions = torch.from_numpy(np.vstack([e.action for e in experiences if e is not None])).double().to(device)
        rewards = torch.from_numpy(np.vstack([e.reward for e in experiences if e is not None])).double().to(device)
        next_states = torch.from_numpy(np.vstack([e.next_state for e in experiences if e is not None])).double().to(device)
        dones = torch.from_numpy(np.vstack([e.done for e in experiences if e is not None])).double().to(device)
        return (states, actions, rewards, next_states, dones)

    # experiences = random.sample(self.memory, k=BATCH_SIZE)

    # batch = self.experiences(*zip(experiences))

    # states = torch.cat(batch.state)
    # actions = torch.cat(batch.actions)
    # rewards = torch.cat(batch.reward)
    # next_states = torch.cat(batch.next_state)
    # dones = torch.cat(batch.done)
    #return random.sample(self.memory, BATCH_SIZE)

    
  
    def __len__(self):
        return len(self.memory)


In [38]:
import json
deep_agent = Agent(action_size=49, seed=0)

state = 0
env.state_idx = 0

true_positive = []
true_negative = []

false_positive = []
false_negative = []

TPR = []
FPR = []

current_state = env.df_xy.iloc[0, :-1].values

RuntimeError: CUDA error: device-side assert triggered

In [25]:
print(len(current_state))
print(current_state)

52
[1.00000000e+00 1.03214180e+07 3.00000000e+00 3.89196528e+03
 3.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 1.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 3.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 1.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 1.00000000e+00 4.00000000e+00
 5.00000000e+00 1.00000000e+00 9.90000000e+01 3.00000000e+01
 9.59828230e+04 1.00000000e+00 4.00000000e+00 1.00000000e+00]


In [26]:
# check reward strategy once
# add probability to epsilon_greedy

for i in range(1):  
    #print("==========================EPOCH {} COMPLETED===================".format(i))  

    print('Current state : ', i)
    score = 0
    print(train_loader)
    for state_idx, data in enumerate(train_loader, 0):
        inputs, labels = data
        #print(inputs.shape)
        action = deep_agent.epsilon_greedy_action(inputs)
        for a in action:
            #print('action taken : ', len(action))
            next_state, reward, done, info = env.step(a)
            next_state = next_state.iloc[:-1].values
            #print('Next state dtype ; ', next_state.dtype)
            #print('Next state ', current_state.shape)
            deep_agent.step(current_state, a, reward, next_state, done)
            current_state = next_state
            #state = next_state
            #score += reward
            #roc_info = json.loads(info)
      
      
     

Current state :  0
Chosing  random actions for the batch :  [37, 20, 12, 25, 19, 38, 14, 23, 28, 44, 24, 13, 37, 30, 12, 44, 48, 39, 44, 15, 35, 44, 33, 23, 4, 21, 29, 44, 47, 23, 42, 12, 39, 26, 0, 35, 19, 40, 32, 0, 24, 42, 11, 15, 42, 9, 27, 11, 47, 39, 21, 3, 15, 24, 45, 5, 27, 34, 26, 39, 26, 47, 29, 28]
Predicted action based on QNetwork :  tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
Experience sampled from memory :  (tensor([[1.8000e+01, 1.0200e+07, 1.0000e+00,  ..., 0.0000e+00, 3.0000e+00,
         1.0000e+00],
        [1.8000e+01, 1.0200e+07, 1.0000e+00,  ..., 0.0000e+00, 3.0000e+00,
         1.0000e+00],
        [1.8000e+01, 1.0035e+07, 7.0000e+00,  ..., 1.0000e+00, 2.0000e+00,
         1.0000e+00],
        ...,
        [1.8000e+01, 1.0062e+07, 2.0000e+00,  ..., 1.0000e+00, 4.0000e+00,
         1.0000e+00]

predicted_targets: tensor([[1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [0.],
        [1.],
        [0.],
        [0.],
        [1.],
        [1.],
        [1.],
        [0.],
        [1.],
        [1.],
        [1.],
        [0.],
        [0.],
        [0.],
        [1.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [1.],
        [1.],
        [1.],
        [1.],
        [0.],
        [0.],
        [1.],
        [1.],
        [1.],
        [0.],
        [0.],
        [1.],
        [0.],
        [1.],
        [0.],
        [0.],
        [0.],
        [1.],
        [1.],
        [0.],
        [1.],
        [1.],
        [0.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [0.],
        [0.],
        [1.],
        [0.],
        [1.],
        [0.],
        [0.],
        [1.],
        [0.],
        [0.],
        [1.]], dtype=torch.float64, grad_fn=<GatherBackward>)
===========================Training l

RuntimeError: all elements of input should be between 0 and 1

In [2]:
df = pd.read_csv('./dataset/tbrain_cc_training_48tags_hash_final.csv')
df.head()

,dt,chid,shop_tag,txn_cnt,txn_amt,domestic_offline_cnt,domestic_online_cnt,overseas_offline_cnt,overseas_online_cnt,domestic_offline_amt_pct,...,masts,educd,trdtp,naty,poscd,cuorg,slam,gender_code,age,primary_card
0,1,10321418,45,3,3891.965283,3,0,0,0,1.00,...,1.0,4.0,5.0,1.0,99.0,30.0,95982.822967,1.0,4.0,1
1,1,10414574,15,2,10616.561549,2,0,0,0,1.00,...,2.0,2.0,15.0,1.0,2.0,30.0,130702.351368,1.0,3.0,1
2,1,10134567,48,2,23527.655416,0,2,0,0,0.00,...,1.0,3.0,11.0,1.0,3.0,30.0,112010.611717,0.0,4.0,1
3,1,10001003,48,9,17751.558260,7,2,0,0,0.75,...,2.0,3.0,11.0,1.0,2.0,30.0,59701.507360,0.0,3.0,1
4,1,10267183,2,1,21701.307598,1,0,0,0,1.00,...,2.0,6.0,15.0,1.0,99.0,30.0,NaN,0.0,6.0,0


In [3]:
shop_tag = df.pop('shop_tag')
df.insert(52,'shop_tag', shop_tag)
df.head()

,dt,chid,txn_cnt,txn_amt,domestic_offline_cnt,domestic_online_cnt,overseas_offline_cnt,overseas_online_cnt,domestic_offline_amt_pct,domestic_online_amt_pct,...,educd,trdtp,naty,poscd,cuorg,slam,gender_code,age,primary_card,shop_tag
0,1,10321418,3,3891.965283,3,0,0,0,1.00,0.00,...,4.0,5.0,1.0,99.0,30.0,95982.822967,1.0,4.0,1,45
1,1,10414574,2,10616.561549,2,0,0,0,1.00,0.00,...,2.0,15.0,1.0,2.0,30.0,130702.351368,1.0,3.0,1,15
2,1,10134567,2,23527.655416,0,2,0,0,0.00,1.00,...,3.0,11.0,1.0,3.0,30.0,112010.611717,0.0,4.0,1,48
3,1,10001003,9,17751.558260,7,2,0,0,0.75,0.25,...,3.0,11.0,1.0,2.0,30.0,59701.507360,0.0,3.0,1,48
4,1,10267183,1,21701.307598,1,0,0,0,1.00,0.00,...,6.0,15.0,1.0,99.0,30.0,NaN,0.0,6.0,0,2


In [4]:
df['shop_tag'].unique()

array(['45', '15', '48', '2', '12', '33', '10', 'other', '36', '6', '37',
       '19', '17', '31', '26', '9', '11', '42', '21', '41', '39', '13',
       '40', '25', '1', '16', '18', '32', '5', '46', '38', '47', '7',
       '43', '22', '14', '30', '4', '35', '44', '34', '20', '23', '24',
       '3', '8', '28', '27', '29'], dtype=object)

In [5]:
df.loc[df.shop_tag == 'other', 'shop_tag']='0'
df['shop_tag'].unique()

array(['45', '15', '48', '2', '12', '33', '10', '0', '36', '6', '37',
       '19', '17', '31', '26', '9', '11', '42', '21', '41', '39', '13',
       '40', '25', '1', '16', '18', '32', '5', '46', '38', '47', '7',
       '43', '22', '14', '30', '4', '35', '44', '34', '20', '23', '24',
       '3', '8', '28', '27', '29'], dtype=object)

In [15]:
df.head()

,dt,chid,txn_cnt,txn_amt,domestic_offline_cnt,domestic_online_cnt,overseas_offline_cnt,overseas_online_cnt,domestic_offline_amt_pct,domestic_online_amt_pct,...,educd,trdtp,naty,poscd,cuorg,slam,gender_code,age,primary_card,shop_tag
0,1,10321418,3,3891.965283,3,0,0,0,1.00,0.00,...,4.0,5.0,1.0,99.0,30.0,95982.822967,1.0,4.0,1,45
1,1,10414574,2,10616.561549,2,0,0,0,1.00,0.00,...,2.0,15.0,1.0,2.0,30.0,130702.351368,1.0,3.0,1,15
2,1,10134567,2,23527.655416,0,2,0,0,0.00,1.00,...,3.0,11.0,1.0,3.0,30.0,112010.611717,0.0,4.0,1,48
3,1,10001003,9,17751.558260,7,2,0,0,0.75,0.25,...,3.0,11.0,1.0,2.0,30.0,59701.507360,0.0,3.0,1,48
4,1,10267183,1,21701.307598,1,0,0,0,1.00,0.00,...,6.0,15.0,1.0,99.0,30.0,NaN,0.0,6.0,0,2


In [ ]:
df = df.fillna(0)
#df.replace(np.nan, 0)
df.head()

In [7]:
df.to_csv("clean.csv", index=False)

In [12]:
if data.loc[0]['shop_tag'] == '45':
    print(666)

666


In [10]:
len(data)

32975653

In [22]:
import torch, torch.nn as nn, torch.nn.functional as F
from torch.autograd import Variable
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import torch.utils.data as data_utils

X = df.iloc[:, :-1].values #每列
y = df.iloc[:, -1].values #最後一類
print(X)
sc = StandardScaler()
X = sc.fit_transform(X)

[[ 0.00000000e+00 -1.35980713e+00 -7.27811733e-02 ...  1.33558377e-01
  -2.10530535e-02  1.49620000e+02]
 [ 0.00000000e+00  1.19185711e+00  2.66150712e-01 ... -8.98309914e-03
   1.47241692e-02  2.69000000e+00]
 [ 1.00000000e+00 -1.35835406e+00 -1.34016307e+00 ... -5.53527940e-02
  -5.97518406e-02  3.78660000e+02]
 ...
 [ 1.72788000e+05  1.91956501e+00 -3.01253846e-01 ...  4.45477214e-03
  -2.65608286e-02  6.78800000e+01]
 [ 1.72788000e+05 -2.40440050e-01  5.30482513e-01 ...  1.08820735e-01
   1.04532821e-01  1.00000000e+01]
 [ 1.72792000e+05 -5.33412522e-01 -1.89733337e-01 ... -2.41530880e-03
   1.36489143e-02  2.17000000e+02]]


In [14]:
print(X)

[[-1.99658302 -0.69424232 -0.04407492 ...  0.33089162 -0.06378115
   0.24496426]
 [-1.99658302  0.60849633  0.16117592 ... -0.02225568  0.04460752
  -0.34247454]
 [-1.99656197 -0.69350046 -0.81157783 ... -0.13713686 -0.18102083
   1.16068593]
 ...
 [ 1.6419735   0.98002374 -0.18243372 ...  0.01103672 -0.0804672
  -0.0818393 ]
 [ 1.6419735  -0.12275539  0.32125034 ...  0.26960398  0.31668678
  -0.31324853]
 [ 1.64205773 -0.27233093 -0.11489898 ... -0.00598394  0.04134999
   0.51435531]]


In [5]:
data.describe()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
count,284807.000000,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,...,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,284807.000000,284807.000000
mean,94813.859575,3.918649e-15,5.682686e-16,-8.761736e-15,2.811118e-15,-1.552103e-15,2.040130e-15,-1.698953e-15,-1.893285e-16,-3.147640e-15,...,1.473120e-16,8.042109e-16,5.282512e-16,4.456271e-15,1.426896e-15,1.701640e-15,-3.662252e-16,-1.217809e-16,88.349619,0.001727
std,47488.145955,1.958696e+00,1.651309e+00,1.516255e+00,1.415869e+00,1.380247e+00,1.332271e+00,1.237094e+00,1.194353e+00,1.098632e+00,...,7.345240e-01,7.257016e-01,6.244603e-01,6.056471e-01,5.212781e-01,4.822270e-01,4.036325e-01,3.300833e-01,250.120109,0.041527
min,0.000000,-5.640751e+01,-7.271573e+01,-4.832559e+01,-5.683171e+00,-1.137433e+02,-2.616051e+01,-4.355724e+01,-7.321672e+01,-1.343407e+01,...,-3.483038e+01,-1.093314e+01,-4.480774e+01,-2.836627e+00,-1.029540e+01,-2.604551e+00,-2.256568e+01,-1.543008e+01,0.000000,0.000000
25%,54201.500000,-9.203734e-01,-5.985499e-01,-8.903648e-01,-8.486401e-01,-6.915971e-01,-7.682956e-01,-5.540759e-01,-2.086297e-01,-6.430976e-01,...,-2.283949e-01,-5.423504e-01,-1.618463e-01,-3.545861e-01,-3.171451e-01,-3.269839e-01,-7.083953e-02,-5.295979e-02,5.600000,0.000000
50%,84692.000000,1.810880e-02,6.548556e-02,1.798463e-01,-1.984653e-02,-5.433583e-02,-2.741871e-01,4.010308e-02,2.235804e-02,-5.142873e-02,...,-2.945017e-02,6.781943e-03,-1.119293e-02,4.097606e-02,1.659350e-02,-5.213911e-02,1.342146e-03,1.124383e-02,22.000000,0.000000
75%,139320.500000,1.315642e+00,8.037239e-01,1.027196e+00,7.433413e-01,6.119264e-01,3.985649e-01,5.704361e-01,3.273459e-01,5.971390e-01,...,1.863772e-01,5.285536e-01,1.476421e-01,4.395266e-01,3.507156e-01,2.409522e-01,9.104512e-02,7.827995e-02,77.165000,0.000000
max,172792.000000,2.454930e+00,2.205773e+01,9.382558e+00,1.687534e+01,3.480167e+01,7.330163e+01,1.205895e+02,2.000721e+01,1.559499e+01,...,2.720284e+01,1.050309e+01,2.252841e+01,4.584549e+00,7.519589e+00,3.517346e+00,3.161220e+01,3.384781e+01,25691.160000,1.000000


In [26]:
import pandas as pd
import numpy as np

df = pd.DataFrame(np.random.randn(3,4),columns=['a','b','c','d'])
print(df) 
c = df.pop('c')             #pop出一个Series
df.insert(2,'c_new',c)   #相当于在第4列的位置，添加列名为c_new的列c
print(df)                      #for test


          a         b         c         d
0  0.501857 -0.844214  0.000010  0.542353
1 -0.313508  0.771012 -1.868091  1.731185
2  1.467678 -0.335677  0.611341  0.047971
          a         b     c_new         d
0  0.501857 -0.844214  0.000010  0.542353
1 -0.313508  0.771012 -1.868091  1.731185
2  1.467678 -0.335677  0.611341  0.047971
